In [1]:
import pandas as pd
import numpy as np

In [2]:
 def export_predictions(schema, set):
    df = pd.DataFrame({
        'predicted_condition': np.load(f'../{schema}/execution_state/predict_{set}/output/y_pred.npy'),
        'condition': np.load(f'../{schema}/execution_state/predict_{set}/output/y_true.npy')
    })
    classes = np.load(f'../{schema}/execution_state/data_preprocess/output/classes.npy')
    class_labels = np.load(f'../{schema}/execution_state/data_preprocess/output/class_labels.npy', allow_pickle=True)
    class_labels = {c: l for c,l in zip(classes, class_labels)}
    for col in ['predicted_condition', 'condition']:
        df.loc[:, col] = df[col].apply(lambda x: class_labels[x])

    df.columns = [f'{schema}__{c}' for c in df.columns]
    return df

In [3]:
schemas = [
    'Migrañas_vs_otras',
    'Migraña_sin_aura_vs_otras',
    # 'Cefalea_secundaria_vs_resto', Esta no porque no se pudo computar por el desbalance
    'Migraña_vs_CTA',
    'Migraña_sin_aura_vs_cefalea_tensional',
    'MSA_vs_cefalea_tensional_vs_CTA'
]
for set in ['test', 'train']:
    df = pd.concat([
        export_predictions(schema, set)
        for schema in schemas
    ], axis=1)
    df.to_csv(f'./predictions_{set}.csv', sep=',', index=False)